<a href="https://colab.research.google.com/github/RockNavamuel/youtube_project/blob/main/YouTube_DB_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [ ]:
def connect_to_db(endpoint, dbname, port, username, password):
  try:
    conn = ps.connect(host = endpoint, database = dbname, user = username, password = password, port = port)
  except ps.OperationalError as e:
    raise e
  else:
    print("Connected!")

  return conn

In [ ]:
## DB credentials
endpoint = 'XXXXX'
dbname = 'postgres'
port = '5432'
username = 'postgres'
password = 'XXXXX'
conn = None

conn = connect_to_db(endpoint, dbname, port, username, password)

Connected!


In [ ]:
df = pd.read_csv('youtube_api.csv', index_col = 0)


In [ ]:
## Create table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    upload_time VARCHAR(255) NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

## Check if video exists
def check_if_video_exists(curr, video_id):
  query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
  curr.execute(query, (video_id,))

  return curr.fetchone() is not None

## Update row
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

## Handling updates to data
def update_db(curr,df):
  tmp_df = pd.DataFrame(columns = ['video_id','video_title','upload_date','upload_time','view_count', 'like_count', 'comment_count'])

  for i, row in df.iterrows():
      if check_if_video_exists(curr, row['video_id']):
        update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count'])
      else:
        tmp_df = pd.concat([tmp_df, pd.DataFrame([row])], ignore_index=True)
  return tmp_df


## Insert
def insert_into_table(curr, video_id, video_title, upload_date, upload_time, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, upload_time,
                        view_count, like_count, comment_count) VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, upload_time, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['upload_time'],
                          row['view_count'], row['like_count'], row['comment_count'])


In [ ]:
curr = conn.cursor()

In [ ]:
## On first run, uncomment below:
# create_table(curr)

In [ ]:
new_vid_df = update_db(curr,df)

In [ ]:
## Data cleaning
new_vid_df2 = new_vid_df.drop_duplicates(keep = 'first')

In [ ]:
append_from_df_to_db(curr,new_vid_df2)

In [ ]:
conn.commit()